In [32]:
import pandas as pd
import numpy as np
import json

In [33]:
def generate_info(dataid, name, type, subtype, subtype2, rate, size, uint,
                  notes):
    data = {
        name: {
            "data_id": dataid,
            "data_name": name,
            "data_type": type,
            "data_subtype1": subtype,
            "data_subtype2": subtype2,
            "data_rate": rate,
            "data_size": size,
            "data_unit": uint,
            "data_notes": notes
        }
    }
    return data


In [34]:
data = pd.read_csv("Failures Modes in MCVT v6 - Communication-Data-Service-8.csv")

In [35]:
# data = pd.read_csv("Failures Modes in MCVT v6 - Failure Modes of MCVT.csv")

# data['shape'] = data.groupby('Component ID').transform('count')['System']

# data['Component Fault'] = data['Component Fault'].astype(str)

# data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']

In [36]:
# const (
# 	SRC_GCC   uint8 = 0
# 	SRC_HMS   uint8 = 1
# 	SRC_STR   uint8 = 2
# 	SRC_SPL   uint8 = 11
# 	SRC_ECLSS uint8 = 5
# 	SRC_PWR   uint8 = 3
# 	SRC_AGT   uint8 = 6
# 	SRC_ING   uint8 = 8
# 	SRC_DTB   uint8 = 9
# 	SRC_EXT   uint8 = 7
# )

In [37]:
trans = {
    'ECLSS': 5,
    'Interior Environment': 8,
    'Power': 3,
    'SPL': 11,
    'Structure': 2,
    'C2/Agents': 6,
    'Communication Network': 10
}

In [38]:
type_trans = {
    'MEAS': 1,
    'FDDMEAS':2,
    'CONTROL':3,
    
}

In [39]:
rate_trans = {
    3: 1000,
    2: 4000,
    8: 200,
    3: 1000,
    5: 200,
    10:1,
    11:1000,
    6:1,
}

In [40]:
result = {}
count = 0
for _, row in data.iterrows():
    count += 1
    try:
        dataid = int(row['Data ID'])
        name = str(row['System/Sub-system/Component']) + '-' + str(dataid)

        dtype = int(type_trans[row['Type']])
        subtype = int(trans[row['System']])
        subtype2 = int(row['ComponentID']) if not pd.isna(row['ComponentID']) else int(0)
        rate = rate_trans[subtype]
        size = int(row['Signal Size']) if not pd.isna(row['Signal Size']) else int(1)
        unit = 'n/a'
        notes = row['Failure Mode'] if isinstance(row['Failure Mode'], str) else 'N/A'
        result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))
    except Exception as err:
        print(err)

cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to 

In [41]:
# result = {}
# count = 0

# for _, row in data.iterrows():
#     count += 1
#     dataid = row['Component ID']
#     name = row['System/Sub-system/Component']
#     dtype = 4 if "Sensor" in row['fault'] else 3
#     subtype = trans[row['System']]
#     subtype2 = 255
#     rate = 1
#     size = row['shape']
#     unit = 'n/a'
#     notes = row['fault']
#     result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))


#     if row['Activity Type'] == 3:
#         dataid = _sensor_id(row['Component ID'])
#         name = row['System/Sub-system/Component'] + "_Measure"
#         dtype = 1 
#         subtype = trans[row['System']]
#         subtype2 = 255
#         rate = 1
#         size = 1
#         unit = 'n/a'
#         notes = row['Component']b
#         result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))

In [31]:
json_string = json.dumps(result)
with open('db_info_v6.json', 'w') as outfile:
    outfile.write(json_string)